In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

/usr/local/lib/python3.4/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.2.2 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
full_train_orig = pd.read_csv("./Data/application_train.csv")
test =  pd.read_csv("./Data/application_test.csv")

In [3]:
bureau = pd.read_csv("./Data/bureau.csv")
bureau_gb = bureau.groupby("SK_ID_CURR")[["DAYS_CREDIT", "CREDIT_DAY_OVERDUE", "CNT_CREDIT_PROLONG", "DAYS_CREDIT_UPDATE"
                             , "AMT_CREDIT_SUM_OVERDUE", "AMT_CREDIT_SUM"]].mean().reset_index()

In [4]:
full_train_orig = pd.merge(full_train_orig, bureau_gb, how="inner", on="SK_ID_CURR")
test = pd.merge(test, bureau_gb, how="inner", on="SK_ID_CURR")

In [5]:
'''
Removing columns with more than 100 null values (filters out 62 columns out of 122 columns)
And not useful (based on EDA) which removes reduces feature size to 43)
'''

null_columns = full_train_orig.columns[full_train_orig.isnull().sum().values > 100].values.tolist()
useless_columns = ["FLAG_DOCUMENT_2", "FLAG_DOCUMENT_4", "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_7"
                  ,'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

In [6]:
### Manually removing columns which doesn't make sense (based on EDA)
full_train = full_train_orig.drop(null_columns+useless_columns, axis = 1)

In [7]:
#cat_columns_all = set(full_train.columns) - set(full_train._get_numeric_data().columns.tolist())

In [8]:
# cat_feats = ["CODE_GENDER", "NAME_CONTRACT_TYPE", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", 
#              "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "OCCUPATION_TYPE",
#              "NAME_HOUSING_TYPE", "FLAG_DOCUMENT_3"]
# num_feats = ["AMT_INCOME_TOTAL", "CNT_CHILDREN", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "DAYS_BIRTH",
#              "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_LAST_PHONE_CHANGE", "AMT_REQ_CREDIT_BUREAU_YEAR", 
#             "REGION_POPULATION_RELATIVE", ]

In [9]:
full_train_y = full_train.TARGET.values
full_train = full_train.drop(["TARGET"], axis = 1)
full_train = full_train.set_index("SK_ID_CURR")
num_feats = full_train._get_numeric_data().columns.values.tolist()
cat_feats = list(set(full_train.columns.values) - set(num_feats))

In [10]:
## Categorical Features - Train
train_cat= full_train[cat_feats]
train_cat = pd.get_dummies(train_cat)

## Numerical Features - Train
train_num = full_train[num_feats]
# train[num_feats].isnull().any() # Checking if the column has any null value
#train_num = train_num.fillna((train_num.mean()), inplace=True)

## Categorical Features - Test
test_cat = test[cat_feats]
test_cat = pd.get_dummies(test_cat)

## Numerical Features - Test
test_num = test[num_feats]
test_num = test_num.fillna(test_num.mean())

# ## Standardize numerical features
# std = StandardScaler().fit(train_num)
# train_num = std.transform(train_num)
# test_num = std.transform(test_num)


In [11]:
full_train_feats = pd.concat([train_num, train_cat], axis=1)
test_feats = pd.concat([test_num, test_cat], axis=1)

In [12]:
full_train_feats = full_train_feats.apply(lambda x: x.fillna(x.mean()),axis=0)
test_feats = test_feats.apply(lambda x: x.fillna(x.mean()),axis=0)

In [13]:
train_X, valid_X, train_y, valid_y = train_test_split(full_train_feats, full_train_y, train_size = 0.8, stratify=full_train_y)

# Random Forest Classifier

In [ ]:
# ### RF classifier
# params_rf={
#     'max_depth': [20, 40, 60], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
#     'n_estimators': [100, 300, 500], #[1000,2000,3000]
# }

# rf_clf = RandomForestClassifier()
# rf = GridSearchCV(rf_clf,
#                   params_rf,
#                   cv=3,
#                   scoring="roc_auc",
#                   n_jobs=1,
#                   verbose=2)
# rf.fit(train_X.drop("CODE_GENDER_XNA", axis=1), train_y)
# best_est_rf = rf.best_estimator_
# print(best_est)

In [ ]:
valid_probs_rf = rf.predict_proba(valid_X.drop("CODE_GENDER_XNA", axis=1))[:,1]
valid_preds_rf = rf.predict(valid_X.drop("CODE_GENDER_XNA", axis=1))

In [ ]:
print(accuracy_score(valid_y, valid_preds_rf))
print(roc_auc_score(valid_y, valid_probs_rf))

# XGboost with Grid Search

In [ ]:
params={
    'max_depth': [3, 5], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'subsample': [0.6, 0.8], #[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    #'colsample_bytree': [0.5, 0.7], #[0.5,0.6,0.7,0.8],
    'n_estimators': [300, 500], #[1000,2000,3000]
    #'reg_alpha': [0.01, 0.05] #[0.01, 0.02, 0.03, 0.04]
    'scale_pos_weight':[1,3,5]
}

xgb_clf = xgb.XGBClassifier(missing=9999999999)
rs = GridSearchCV(xgb_clf,
                  params,
                  cv=3,
                  scoring="roc_auc",
                  n_jobs=1,
                  verbose=2)
rs.fit(train_X.drop("CODE_GENDER_XNA", axis=1), train_y)
best_est = rs.best_estimator_
print(best_est)

In [ ]:
# xgb_single = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
#        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
#        min_child_weight=1, missing=9999999999, n_estimators=300,
#        nthread=-1, objective='binary:logistic', reg_alpha=0.05,
#        reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
#        subsample=0.8)

# xgb_single.fit(train_X, train_y)
# valid_probs_xgb_single = xgb_single.predict_proba(valid_X)[:,1]
# valid_preds_xgb_single = xgb_single.predict(valid_X)
# print(accuracy_score(valid_y, valid_preds_xgb_single))
# print(roc_auc_score(valid_y, valid_probs_xgb_single))

In [ ]:
valid_probs_rs = rs.predict_proba(valid_X.drop("CODE_GENDER_XNA", axis=1))[:,1]
valid_preds_rs= rs.predict(valid_X.drop("CODE_GENDER_XNA", axis=1))
print(accuracy_score(valid_y, valid_preds_rs))
print(roc_auc_score(valid_y, valid_probs_rs))

# LightGBM model 

In [19]:
params={
    'max_depth': [2, 3, 5], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'subsample': [0.4, 0.6], #[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.5, 0.7], #[0.5,0.6,0.7,0.8],
    'n_estimators': [300, 500, 700, 1000], #[1000,2000,3000]
    'reg_alpha': [0.01, 0.05, 0.1], #[0.01, 0.02, 0.03, 0.04]
    'scale_pos_weight':[1,3,5], 
    'num_leaves':[10, 30, 50]
    
}

lgb_clf = lgb.LGBMClassifier()
rs = GridSearchCV(lgb_clf,
                  params,
                  cv=3,
                  scoring="roc_auc",
                  n_jobs=1,
                  verbose=2)
rs.fit(train_X.drop("CODE_GENDER_XNA", axis=1), train_y)
best_est = rs.best_estimator_
print(best_est)

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   4.2s
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s remaining:    0.0s


[CV]  subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   4.9s
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   4.6s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   5.3s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   7.4s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, sc

[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05, total=   5.3s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05, total=   6.6s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05, total=   6.5s
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.1 
[CV]  subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.1, total=   9.1s
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.5, max_depth=2, scal

[CV]  subsample=0.6, num_leaves=30, colsample_bytree=0.5, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.01, total=   6.8s
[CV] subsample=0.4, num_leaves=30, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01, total=   7.2s
[CV] subsample=0.4, num_leaves=30, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01, total=   8.9s
[CV] subsample=0.4, num_leaves=30, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.5, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01, total=   5.2s
[CV] subsample=0.6, num_leaves=30, colsample_bytree=0.5, max_depth=2, sc

...

[CV]  subsample=0.4, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   5.7s
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   5.8s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   5.8s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.01, total=   5.7s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, sc

[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05, total=   5.5s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05, total=   5.6s
[CV] subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05 
[CV]  subsample=0.6, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.05, total=   5.6s
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.1 
[CV]  subsample=0.4, num_leaves=10, colsample_bytree=0.7, max_depth=2, scale_pos_weight=1, n_estimators=300, reg_alpha=0.1, total=   5.8s
[CV] subsample=0.4, num_leaves=10, colsample_bytree=0.7, max_depth=2, scal

[CV]  subsample=0.6, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.01, total=   5.7s
[CV] subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01, total=   5.6s
[CV] subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01, total=   5.9s
[CV] subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01 
[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=5, n_estimators=300, reg_alpha=0.01, total=   5.6s
[CV] subsample=0.6, num_leaves=30, colsample_bytree=0.7, max_depth=2, sc

[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.1, total=   5.9s
[CV] subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.1 
[CV]  subsample=0.4, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.1, total=   5.7s
[CV] subsample=0.6, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.1 
[CV]  subsample=0.6, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.1, total=   6.1s
[CV] subsample=0.6, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.1 
[CV]  subsample=0.6, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos_weight=3, n_estimators=300, reg_alpha=0.1, total=   5.7s
[CV] subsample=0.6, num_leaves=30, colsample_bytree=0.7, max_depth=2, scale_pos

KeyboardInterrupt: 

In [15]:
valid_probs_rs = rs.predict_proba(valid_X.drop("CODE_GENDER_XNA", axis=1))[:,1]
valid_preds_rs= rs.predict(valid_X.drop("CODE_GENDER_XNA", axis=1))
print(accuracy_score(valid_y, valid_preds_rs))
print(roc_auc_score(valid_y, valid_probs_rs))

0.922655078844
0.700247442872


In [16]:
rs.best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=3, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=500,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, scale_pos_weight=1, silent=True,
        subsample=0.6, subsample_for_bin=200000, subsample_freq=0)

# Prepare Submission file 

In [ ]:
### Prepare submission file and save to disk
result_df = pd.DataFrame({'SK_ID_CURR':test.SK_ID_CURR.values, "TARGET":rs.predict_proba(test_feats)[:,1]})
result_df.to_csv("submimssion_2_xgboost_gridsearch_allfeatures_withweight.csv", index=False)